# 單天版 -- 將網頁寫入txt裡

In [ ]:
import time
import requests
from bs4 import BeautifulSoup as bs

with open('ltn.txt', 'w') as f:
    url = 'http://news.ltn.com.tw/newspaper/focus/20160310'
    res = requests.get(url)
    soup = bs(res.text)

    Times = 'http://news.ltn.com.tw'
    sort = soup.select('.newspaper a')
    for a in xrange(0, 12):
        url2 = Times + sort[a]['href'] #分類網址
        res2 = requests.get(url2)
        soup2 = bs(res2.text)
        print len(soup2.select('.p_num'))
        if len(soup2.select('.p_num')) == 1 : #判斷此類別新聞是否一頁
            title = soup2.select('.picword')
            title_num = len(soup2.select('.picword')) #計算有幾篇文章(每天新聞數量不同)
            for i in xrange(0 , title_num):   
                url1=Times+title[i]['href']   #文章網址
                f.write(url1  + '\n')  #寫入text
        else:
            for page1 in xrange (1,len(soup2.select('.p_num'))+1): #此類別新聞超過一頁時執行
                url3= url2+'?page={}'.format(page1)
                print url3
                res3 = requests.get(url3)
                soup3 = bs(res3.text)
                title = soup3.select('.picword')
                title_num1 = len(soup3.select('.picword')) #計算有幾篇文章(每天新聞數量不同)
                for i in xrange(0 , title_num1):   
                    url1=Times+title[i]['href']   #文章網址
                    f.write(url1  + '\n')  #寫入text

# 單天版-- 讀取txt裡網頁 並寫成檔案

In [10]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import sys

reload(sys)  
sys.setdefaultencoding("utf-8")   #解決UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-6:ordinal not in range(128) 

def wr_all(url):  #方法 大部分的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category = soup.select('.guide  a ')
    keyword= soup.select('.con_keyword')
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文       
    return tital[0].text,date[0].text,category[1].text, page
        
def wr_sport(url):  #方法 運動類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)    
    essay = soup.select('h4 , p')
    tital = soup.select('.Btitle ')
    date =  soup.select('.news_content .date ')
    category = "體育"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)     
    return tital[0].text,date[0].text,category, page
     
def wr_ent(url):  #方法 娛樂類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('p')
    tital = soup.select('.news_content h1 ')
    date =  soup.select('.news_content .date')
    category = "娛樂"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)    
    return tital[0].text,date[0].text,category, page

def wr_local(url): #方法 地方類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category1 = "地方"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)
    return tital[0].text,date[0].text,category1, page
    
def wr_talk(url): # 方法 言論類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('.cont p')
    tital = soup.select('h2 ')
    date =  soup.select('.writer span')
    category = "言論"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)    
    return tital[0].text,date[0].text,category, page
        
    
        
with open('ltn.txt', 'r') as f:
    for line in f.readlines():
        url = line.strip()
        m = re.match('.*news/(\w+)/paper/(\w+)' ,url)  #正規表達法 切出m.group(1)類別 與 m.group(2)網頁最後的數字(用來當檔名)
        print m.group(1)
        if m.group(1) == 'sports': #判斷是否為體育類
            x=  wr_sport(url) #使用方法
            #print x[0],x[1],x[2]
            name =  m.group(2)
            f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            iii_num = len(x)
            for iii in xrange (0, iii_num):
                f.write( x[iii] + '\n') #寫入內文      
            f.close()
            time.sleep(1)
            
        elif m.group(1) == 'entertainment': #判斷是否為娛樂類
            x= wr_ent(url) #使用方法
            #print x[0],x[1],x[2]
            name =  m.group(2)
            f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            iii_num = len(x) #計算陣列長度
            for iii in xrange (0, iii_num):
                f.write( x[iii] + '\n') #寫入內文           
            f.close()
            time.sleep(1)
            
        elif m.group(1) =='local': #判斷是否為地方類
            x= wr_local(url) #使用方法
            #print x[0],x[1],x[2]
            name =  m.group(2)
            f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            iii_num = len(x)  
            for iii in xrange (0, iii_num):
                f.write( x[iii] + '\n') #寫入內文         
            f.close()
            time.sleep(1)
            
        elif m.group(1) =='opinion': #判斷是否為言論類
            x= wr_talk(url) #使用方法
            #print x[0],x[1],x[2]
            name =  m.group(2)
            f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            iii_num = len(x)
            for iii in xrange (0, iii_num):
                f.write( x[iii] + '\n') #寫入內文          
            f.close()
            time.sleep(1)
            
        else:
            x= wr_all(url) #使用方法
            #print x[0],x[1],x[2]
            name =  m.group(2)
            f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            iii_num = len(x)
            for iii in xrange (0, iii_num):
                f.write( x[iii] + '\n') #寫入內文        
            f.close()
            time.sleep(1)
            

KeyboardInterrupt: 

# 全年無休版--將網頁寫入txt裡

In [15]:
-

# 全年無休版--讀取txt裡網頁 並寫成檔案

In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import sys
import datetime as dt

reload(sys)  
sys.setdefaultencoding("utf-8")   #解決UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-6:ordinal not in range(128) 



def wr_all(url):  #方法 大部分的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category = soup.select('.guide  a ')
    keyword= soup.select('.con_keyword')
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文       
    return tital[0].text,date[0].text,category[1].text, page
        
def wr_sport(url):  #方法 運動類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)    
    essay = soup.select('h4 , p')
    tital = soup.select('.Btitle ')
    date =  soup.select('.news_content .date ')
    category = "體育"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)     
    return tital[0].text,date[0].text,category, page
     
def wr_ent(url):  #方法 娛樂類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('p')
    tital = soup.select('.news_content h1 ')
    date =  soup.select('.news_content .date')
    category = "娛樂"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)    
    return tital[0].text,date[0].text,category, page

def wr_local(url): #方法 地方類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category1 = "地方"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)
    return tital[0].text,date[0].text,category1, page
    
def wr_talk(url): # 方法 言論類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('.cont p')
    tital = soup.select('h2 ')
    date =  soup.select('.writer span')
    category = "言論"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文
        time.sleep(1)    
    return tital[0].text,date[0].text,category, page
        
    
startdate = dt.datetime(2016, 3,9)  #輸入開始日期
endate = dt.datetime(2016, 3,12)       #輸入結束日期
totaldays = (endate - startdate).days + 1  #計算有幾天
for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d")  
        
    with open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(datestring), 'r') as f:
        for line in f.readlines():
            url = line.strip()
            m = re.match('.*news/(\w+)/paper/(\w+)' ,url)  #正規表達法 切出m.group(1)類別 與 m.group(2)網頁最後的數字(用來當檔名)
            print m.group(1)
            if m.group(1) == 'sports': #判斷是否為體育類
                x=  wr_sport(url) #使用方法
                #print x[0],x[1],x[2]
                name =  m.group(2)
                f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
                iii_num = len(x)
                for iii in xrange (0, iii_num):
                    f.write( x[iii] + '\n') #寫入內文      
                f.close()
                time.sleep(1)

            elif m.group(1) == 'entertainment': #判斷是否為娛樂類
                x= wr_ent(url) #使用方法
                #print x[0],x[1],x[2]
                name =  m.group(2)
                f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
                iii_num = len(x) #計算陣列長度
                for iii in xrange (0, iii_num):
                    f.write( x[iii] + '\n') #寫入內文           
                f.close()
                time.sleep(1)

            elif m.group(1) =='local': #判斷是否為地方類
                x= wr_local(url) #使用方法
                #print x[0],x[1],x[2]
                name =  m.group(2)
                f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
                iii_num = len(x)  
                for iii in xrange (0, iii_num):
                    f.write( x[iii] + '\n') #寫入內文         
                f.close()
                time.sleep(1)

            elif m.group(1) =='opinion': #判斷是否為言論類
                x= wr_talk(url) #使用方法
                #print x[0],x[1],x[2]
                name =  m.group(2)
                f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
                iii_num = len(x)
                for iii in xrange (0, iii_num):
                    f.write( x[iii] + '\n') #寫入內文          
                f.close()
                time.sleep(1)

            else:
                x= wr_all(url) #使用方法
                #print x[0],x[1],x[2]
                name =  m.group(2)
                f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
                iii_num = len(x)
                for iii in xrange (0, iii_num):
                    f.write( x[iii] + '\n') #寫入內文        
                f.close()
                time.sleep(1)


IOError: [Errno 2] No such file or directory: 'C:/Users/BIG DATA/Desktop/python/essay/20160312.txt'

# 寫進單一檔案  new 3/14(修正版)

In [5]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import sys
import datetime as dt

reload(sys)  
sys.setdefaultencoding("utf-8")   #解決UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-6:ordinal not in range(128) 



def wr_all(url):  #方法 大部分的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category = soup.select('.guide  a ')
    keyword= soup.select('.con_keyword')
    photo = soup.select('#newsphoto p')
    page=""                          #給一個字串,存放內文用           
    if len(photo)==0 :
        for div3 in essay[0:(len(essay))]:
            page += div3.text       #加入內文
    else:
        for div3 in essay[0:(len(essay)-len(photo))]:
            page += div3.text
    return tital[0].text,date[0].text,category[1].text, page
        
def wr_sport(url):  #方法 運動類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)    
    essay = soup.select('h4 , p')
    tital = soup.select('.Btitle ')
    date =  soup.select('.news_content .date ')
    category = "體育"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[1:(len(essay))]:
        page += div3.text       #加入內文     
    return tital[0].text,date[0].text,category, page
     
def wr_ent(url):  #方法 娛樂類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('p')
    tital = soup.select('.news_content h1 ')
    date =  soup.select('.news_content .date')
    category = "娛樂"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[1:(len(essay))]:
        page += div3.text       #加入內文    
    return tital[0].text,date[0].text,category, page

def wr_local(url): #方法 地方類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    photo = soup.select('#newsphoto p')
    category1 = "地方"
    page=""#給一個字串,存放內文用 
    if len(photo)==0 :
        for div3 in essay[0:(len(essay))]:
            page += div3.text       #加入內文
    else:
        for div3 in essay[0:(len(essay)-len(photo)-len(photo))]:
            page += div3.text
    return tital[0].text,date[0].text,category1, page
    
def wr_talk(url): # 方法 言論類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('.cont p')
    tital = soup.select('h2 ')
    date =  soup.select('.writer span')
    category = "言論"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文   
    return tital[0].text,date[0].text,category, page
        
    
startdate = dt.datetime(2016, 3,9)  #輸入開始日期
endate = dt.datetime(2016, 3,12)       #輸入結束日期
totaldays = (endate - startdate).days + 1  #計算有幾天
for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d")  
    
    with open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(datestring), 'r') as f:
        a = open('C:/Users/BIG DATA/Desktop/python/essay/ltn.txt','w')
        for line in f.readlines():
            url = line.strip()
            m = re.match('.*news/(\w+)/paper/(\w+)' ,url)  #正規表達法 切出m.group(1)類別 與 m.group(2)網頁最後的數字(用來當檔名)
            # print m.group(1)
            if m.group(1) == 'sports': #判斷是否為體育類
                x=  wr_sport(url) #使用方法
                #print x[0],x[1],x[2]
                a.write( x[0] + '#'+x[1] + '#'+ x[2] + '#'+ x[3]+'\n' ) #寫入內文      
                time.sleep(0.5)

            elif m.group(1) == 'entertainment': #判斷是否為娛樂類
                x= wr_ent(url) #使用方法
                #print x[0],x[1],x[2]
                a.write( x[0] + '#'+x[1] + '#'+ x[2] + '#'+ x[3]+'\n' ) #寫入內文      
                time.sleep(0.5)

            elif m.group(1) =='local': #判斷是否為地方類
                x= wr_local(url) #使用方法
                #print x[0],x[1],x[2]
                a.write( x[0] + '#'+x[1] + '#'+ x[2] + '#'+ x[3]+'\n'  ) #寫入內文      
                time.sleep(0.5)

            elif m.group(1) =='opinion': #判斷是否為言論類
                x= wr_talk(url) #使用方法
                #print x[0],x[1],x[2]
                a.write( x[0] + '#'+x[1] + '#'+ x[2] + '#'+ x[3]+'\n'  ) #寫入內文      
                time.sleep(0.5)

            else:
                x= wr_all(url) #使用方法
                #print x[0],x[1],x[2]
                a.write( x[0] + '#'+x[1] + '#'+ x[2] + '#'+ x[3]+'\n'  ) #寫入內文      
                time.sleep(0.5)
        a.close()


IOError: [Errno 2] No such file or directory: 'C:/Users/BIG DATA/Desktop/python/essay/20160310.txt'

# json

In [ ]:
import time
import requests
from bs4 import BeautifulSoup as bs

with open('ltn.txt', 'w') as f:
    url = 'http://news.ltn.com.tw/newspaper/focus/20160310'
    res = requests.get(url)
    soup = bs(res.text)

    Times = 'http://news.ltn.com.tw'
    sort = soup.select('.newspaper a')
    for a in xrange(0, 12):
        url2 = Times + sort[a]['href'] #分類網址
        res2 = requests.get(url2)
        soup2 = bs(res2.text)
        print len(soup2.select('.p_num'))
        if len(soup2.select('.p_num')) == 1 : #判斷此類別新聞是否一頁
            title = soup2.select('.picword')
            title_num = len(soup2.select('.picword')) #計算有幾篇文章(每天新聞數量不同)
            for i in xrange(0 , title_num):   
                url1=Times+title[i]['href']   #文章網址
                f.write(url1  + '\n')  #寫入text
        else:
            for page1 in xrange (1,len(soup2.select('.p_num'))+1): #此類別新聞超過一頁時執行
                url3= url2+'?page={}'.format(page1)
                print url3
                res3 = requests.get(url3)
                soup3 = bs(res3.text)
                title = soup3.select('.picword')
                title_num1 = len(soup3.select('.picword')) #計算有幾篇文章(每天新聞數量不同)
                for i in xrange(0 , title_num1):   
                    url1=Times+title[i]['href']   #文章網址
                    f.write(url1  + '\n')  #寫入text
        

In [4]:
# 標題,類別,內文,關鍵詞,新聞連結網址,日期(格式yyyyMMdd),點擊次數

import re
import time
import requests
from bs4 import BeautifulSoup as bs
import sys
import json
import ast  #轉換成json需要套件


#reload(sys)  
#sys.setdefaultencoding("utf-8")   #解決UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-6:ordinal not in range(128) 

def wr_all(url):  #方法 大部分的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category = soup.select('.guide  a ')
    keyword= soup.select('.con_keyword')
    #page=""                          #給一個字串,存放內文用           
    #for div3 in essay[0:(len(essay))]:
        #page += div3.text       #加入內文       
    return tital[0].text,date[0].text,category[1].text, #page
        
def wr_sport(url):  #方法 運動類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)    
    essay = soup.select('h4 , p')
    tital = soup.select('.Btitle ')
    date =  soup.select('.news_content .date ')
    category = "體育"
    #page=""                          #給一個字串,存放內文用           
    #for div3 in essay[0:(len(essay))]:
        #page += div3.text       #加入內文
        #time.sleep(1)     
    return tital[0].text,date[0].text,category, #page
     
def wr_ent(url):  #方法 娛樂類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('p')
    tital = soup.select('.news_content h1 ')
    date =  soup.select('.news_content .date')
    category = "娛樂"
    #page=""                          #給一個字串,存放內文用           
    #for div3 in essay[0:(len(essay))]:
        #page += div3.text       #加入內文
        #time.sleep(1)    
    return tital[0].text,date[0].text,category, #page

def wr_local(url): #方法 地方類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')
    date =  soup.select('#newstext span ')
    category1 = "地方"
    #page=""                          #給一個字串,存放內文用           
    #for div3 in essay[0:(len(essay))]:
        #page += div3.text       #加入內文
        #time.sleep(1)
    return tital[0].text,date[0].text,category1, #page
    
def wr_talk(url): # 方法 言論類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('.cont p')
    tital = soup.select('h2 ')
    date =  soup.select('.writer span')
    category = "言論"
    #page=""                          #給一個字串,存放內文用           
    #for div3 in essay[0:(len(essay))]:
        #page += div3.text       #加入內文
        #time.sleep(1)    
    return tital[0].text,date[0].text,category, #page
        
    
        
with open('ltn.txt', 'r') as f:
    for line in f.readlines():
        url = line.strip()
        m = re.match('.*news/(\w+)/paper/(\w+)' ,url)  #正規表達法 切出m.group(1)類別 與 m.group(2)網頁最後的數字(用來當檔名)
        print m.group(1)
        if m.group(1) == 'sports': #判斷是否為體育類
            x=  wr_sport(url) #使用方法
            print x[0],x[1],x[2]
            #name =  m.group(2)
            #f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            #iii_num = len(x)
            #for iii in xrange (0, iii_num):
                #f.write( x[iii] + '\n') #寫入內文      
            #f.close()
            #time.sleep(1)
            
        elif m.group(1) == 'entertainment': #判斷是否為娛樂類
            x= wr_ent(url) #使用方法
            print x[0],x[1],x[2]
            #name =  m.group(2)
            #f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            #iii_num = len(x) #計算陣列長度
            #for iii in xrange (0, iii_num):
              #  f.write( x[iii] + '\n') #寫入內文           
            #f.close()
            #time.sleep(1)
            
        elif m.group(1) =='local': #判斷是否為地方類
            x= wr_local(url) #使用方法
            print x[0],x[1],x[2]
            #name =  m.group(2)
            #f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            #iii_num = len(x)  
            #for iii in xrange (0, iii_num):
                #f.write( x[iii] + '\n') #寫入內文         
            #f.close()
            #time.sleep(1)
            
        elif m.group(1) =='opinion': #判斷是否為言論類
            x= wr_talk(url) #使用方法
            print x[0],x[1],x[2]
            #name =  m.group(2)
            #f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            #iii_num = len(x)
            #for iii in xrange (0, iii_num):
                #f.write( x[iii] + '\n') #寫入內文          
            #f.close()
            #time.sleep(1)
            
        else:
            x= wr_all(url) #使用方法
            print x[0],x[1],x[2]
            #name =  m.group(2)
            #f = open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(name),'w')
            #iii_num = len(x)
            #for iii in xrange (0, iii_num):
                #f.write( x[iii] + '\n') #寫入內文        
            #f.close()
            #time.sleep(1)
            

In [3]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs


url  = 'http://news.ltn.com.tw/news/politics/breakingnews/1638642'

res = requests.get(url)
res.encoding = "UTF-8"
soup = bs(res.text)
essay = soup.select('#newstext p , h4')
tital = soup.select('h1 ')
date =  soup.select('#newstext span ')
category = soup.select('.guide  a ')
keyword= soup.select('.con_keyword')
page=""                          #給一個字串,存放內文用           
for div3 in essay[0:(len(essay))]:
        page += div3.text       #加入內文       
print tital[0].text,date[0].text,category[1].text, page




醫院評鑑如同高裝檢 段宜康：哪有不造假的？ 2016-03-20  21:19 政治 〔即時新聞／綜合報導〕衛生福利部每年都會委託醫策會，對全台各醫院進行評鑑，引來廣大醫護人員不滿，民進黨立委段宜康對此表示，醫院評鑑如同高裝檢，而「高裝檢哪有不造假的？」段宜康在臉書上轉貼醫師洪浩雲的PO文指出，「衛福部大概沒有人當過兵」，所有當過兵的民眾心裡都明白「高裝檢哪有不造假的？」，而醫院的評鑑制度就如同軍中的高裝檢，衛福部每年委託醫策會到醫院打分數，「先通知要準備哪些資料、看哪些設備」。段宜康批評，若是米其林評鑑也使用如同高裝檢般「官樣文章」，「一定成為笑柄」，衛福部對於我國的醫院評鑑，「不學舉世信服的米其林；去學了自欺欺人的高裝檢」；醫師洪浩雲亦認為，政府官員如果覺得醫院評鑑相當重要，應重視現實層面的問題。段宜康臉書全文衞福部大概沒有人當過兵。當過兵的人都知道：高裝檢哪有不造假的？衛福部委託醫策會，每年固定時間、排定時程來檢查醫院、打分數。先通知要準備哪些資料、看哪些設備。各位後備軍人和現役阿兵哥：是不是很熟悉？醫院評鑑就是高裝檢！米...民進黨立委段宜康表示，醫院評鑑如同高裝檢，而「高裝檢哪有不造假的？」。（資料照，記者羅沛德攝）
